# Collaboration and Competition

## 1. Start the Environment

In [ ]:
import gym
import numpy as np
import random
import torch
import time # for perf count

from collections import deque
from unityagents import UnityEnvironment
import matplotlib.pyplot as plt
%matplotlib inline

Next, we will start the environment! Before running the code cell below, change the file_name parameter to match the location of the Unity environment that you downloaded.

- Mac: "path/to/Tennis.app"
- Windows (x86): "path/to/Tennis_Windows_x86/Tennis.exe"
- Windows (x86_64): "path/to/Tennis_Windows_x86_64/Tennis.exe"
- Linux (x86): "path/to/Tennis_Linux/Tennis.x86"
- Linux (x86_64): "path/to/Tennis_Linux/Tennis.x86_64"
- Linux (x86, headless): "path/to/Tennis_Linux_NoVis/Tennis.x86"
- Linux (x86_64, headless): "path/to/Tennis_Linux_NoVis/Tennis.x86_64"

For instance, if you are using a Mac, then you downloaded Tennis.app. If this file is in the same folder as the notebook, then the line below should appear as follows:

env = UnityEnvironment(file_name="Tennis.app")

In [ ]:
env = UnityEnvironment(file_name="unity_env/Tennis.x86_64")

Environments contain brains which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

## 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1. If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01. Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping.

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

## 3. Train Agents with MADDPG

### 3.1 Instantiate Agents

#### 3.1.1 Create the Shared Replay Buffer

In MADDPG, all of the agents share the same Replay Buffer. Create the buffer here once and then pass it to the agents.

In [ ]:
from maddpg_agent import Agent, ReplayBuffer
from maddpg_agent import BUFFER_SIZE, BATCH_SIZE

random_seed = 12

In [ ]:
shared_replay_buffer = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, random_seed)

#### 3.1.2 Create Agents

In [ ]:
agents = [None] * num_agents
for i in range(num_agents):
    agents[i] = Agent(state_size=state_size, action_size=action_size, random_seed=random_seed, replay_buffer=shared_replay_buffer)

In [ ]:
def forall(seq, f):
    for item in seq:
        f(item)

In [ ]:
def maddpg(n_episodes=2000, max_t=10000):
    scores_deque = deque(maxlen=100)
    scores = []
    max_score = -np.Inf
    for i_episode in range(1, n_episodes+1):
        start_time = time.perf_counter()
        env_info = env.reset(train_mode=True)[brain_name]
        states = env_info.vector_observations
        forall(agents, lambda agent: agent.reset())
        episode_scores = np.zeros(num_agents)
        for t in range(max_t):
            actions = [agent.act(state) for agent, state in zip(agents, states)]
            env_info = env.step(actions)[brain_name]
            next_states = env_info.vector_observations
            rewards = env_info.rewards
            dones = env_info.local_done
            
            for i in range(num_agents):
                agents[i].step(states[i], actions[i], rewards[i], next_states[i], dones[i])
            
            states = next_states
            episode_scores += rewards
            if np.any(dones):
                break 
        score = max(episode_scores)
        scores_deque.append(score)
        scores.append(score)
        end_time = time.perf_counter()
        print('\rEpisode {}\tAverage Score: {:.2f}\tScore: {:.2f} \tTime/Episode: {:.2f}'.format(i_episode, np.mean(scores_deque), score, (end_time - start_time)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f} \tTime/Episode: {:.2f}'.format(i_episode, np.mean(scores_deque), (end_time - start_time)))
            
        if np.mean(scores_deque) >= 0.5:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_deque)))
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            break
    return scores

In [ ]:
scores = maddpg(100)

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

## 4. Watch Smart Agents

In [ ]:
agent.actor_local.load_state_dict(torch.load('checkpoint_actor.pth'))

for i in range(1, 6):
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    state = env_info.vector_observations[0]                # get the current state (for each agent)
    score = np.zeros(num_agents)                           # initialize the score (for each agent)
    while True:
        action = agent.act(state)                          # select an action (for each agent)
        action = np.clip(action, -1, 1)                    # all actions between -1 and 1
        env_info = env.step(action)[brain_name]            # send all actions to tne environment
        next_state = env_info.vector_observations[0]       # get next state (for each agent)
        reward = env_info.rewards[0]                       # get reward (for each agent)
        done = env_info.local_done[0]                      # see if episode finished
        score += env_info.rewards[0]                       # update the score (for each agent)
        state = next_state                                 # roll over states to next time step
        if done:                                           # exit loop if episode finished
            break
    print('Total score (averaged over agents) this episode: {}'.format(score))

## 5. Cleanup

When finished, you can close the environment.

In [ ]:
env.close()